# Day 2 - Self-Paced Practice: Designing a Real-Time Chat Application

**Objective:** Reinforce the concept of diagram-as-code by generating architectural diagrams for a new problem domain.

**Estimated Time:** 45 minutes

**Introduction:**
A key skill for a modern architect is the ability to quickly visualize and communicate system design. In this lab, you will practice using an LLM to generate two different types of diagrams for a common software system: a real-time chat application. This will help you become more fluent in prompting for and interpreting diagrams-as-code.

## 1. Setup

We'll begin by setting up our environment, importing the necessary utilities and initializing our LLM client.

In [1]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, render_plantuml_diagram, save_artifact, clean_llm_output

# For design tasks, models with strong reasoning are recommended.
client, model_name, api_provider = setup_llm_client(model_name="gpt-4.1")

2025-11-05 15:32:51,902 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4.1 latency_ms=None artifacts_path=None


## 2. The Problem Statement

Your task is to design the high-level architecture for a simple, real-time chat application. 

**Core Components & Flow:**
1.  A **Frontend** (web browser) where users interact.
2.  A **Backend API Server** to handle user authentication and message history.
3.  A **WebSocket Server** to handle real-time message broadcasting.
4.  A **Database** to store user information and chat messages.

### ⭐ Deeper Dive: The Power of "Diagrams as Code"

The technique you're using in this lab—generating PlantUML or Mermaid code from a prompt—is a form of **"Diagrams as Code."** This is a powerful, modern practice with several key advantages over using traditional drag-and-drop GUI tools:

- **Version Controllable:** Since the diagram is just a text file, you can check it into Git, review changes in pull requests, and track its history alongside your application code.
- **Easily Editable:** Modifying a relationship or adding a component is as simple as changing a line of text. This is often much faster than fiddling with boxes and arrows in a GUI.
- **Consistent:** Templates and code generation ensure that all diagrams across your organization can share a consistent style and level of detail.
- **Automatable:** Because it's text, you can build tools that automatically generate or update architecture diagrams as part of your CI/CD pipeline, ensuring your documentation never goes stale.

## 3. Your Tasks

### Task 1: Generate a Component Diagram

First, we'll create a **component diagram**. This type of diagram is excellent for showing the high-level static structure of a system and how the major pieces are connected.

#### 💡 Pro-Tip: Prompting for Structural Diagrams

When prompting an LLM to generate a structural diagram (like a Component, C4, or Class diagram), follow this pattern for the best results:

1.  **State the Format:** Begin by clearly stating you want `PlantUML code for a component diagram`.
2.  **Define the Components:** Explicitly list the key components or containers you want to see in the diagram. This gives the LLM the primary "actors."
3.  **Describe the Relationships:** Clearly describe the connections *between* the components. Use directional language like "Component A sends data to Component B via HTTPS."
4.  **Constrain the Output:** End by reminding the model to `"Output only the raw PlantUML code."`

In [4]:
# Load context from PRD and database schema
from utils import load_artifact

# Load the PRD content and schema to provide context
prd_content = load_artifact("artifacts/prd_gen.md")
schema_content = load_artifact("artifacts/schema.sql")

# TODO: Write a prompt to generate a PlantUML component diagram.
# Updated to be specific to our PRD production application
component_diagram_prompt = f"""
You are an expert system architect. Generate PlantUML code for a component diagram that describes the AI-Powered Requirement Analyzer application based on the provided PRD and database schema.

CONTEXT - Product Requirements Document:
{prd_content}

CONTEXT - Database Schema:
{schema_content}

Based on the PRD and schema, the application is an intelligent web platform that transforms problem statements into comprehensive Product Requirements Documents using AI analysis.

The diagram should include these specific components for the AI-Powered Requirement Analyzer:
- A 'Web Frontend' (React/Vue.js interface for user interaction)
- A 'FastAPI Backend Server' (handles REST API endpoints, file uploads, user management)
- A 'AI Processing Engine' (manages LLM interactions, requirement analysis, PRD generation)
- A 'Background Job Queue' (handles async AI processing tasks)
- A 'SQLite Database' (stores conversations, messages, PRD versions, file attachments)
- A 'File Storage System' (manages uploaded documents and generated exports)
- A 'External AI APIs' (OpenAI/Anthropic/Google for LLM processing)

The relationships should reflect the actual application flow:
- The Web Frontend communicates with the FastAPI Backend Server over HTTPS for user authentication, conversation management, and PRD operations
- The FastAPI Backend Server stores and retrieves conversation data, messages, and PRD versions from the SQLite Database
- The FastAPI Backend Server queues AI processing tasks to the Background Job Queue for async operations
- The Background Job Queue delegates AI analysis to the AI Processing Engine
- The AI Processing Engine calls External AI APIs to perform requirement analysis and PRD generation
- The FastAPI Backend Server manages file uploads and exports through the File Storage System
- The AI Processing Engine updates conversation status and stores results back to the SQLite Database via the FastAPI Backend Server

Output only the raw PlantUML code inside a markdown block.
"""

print("--- Generating Component Diagram for AI-Powered Requirement Analyzer ---")
component_puml_raw = get_completion(component_diagram_prompt, client, model_name, api_provider)
component_puml = clean_llm_output(component_puml_raw, language='plantuml')

print("\n--- Generated PlantUML Code ---")
print(component_puml)

# Save the PlantUML code to a file for manual rendering if needed
if component_puml:
    save_artifact(component_puml, "artifacts/prd_analyzer_component_diagram.puml", overwrite=True)
    print("\n✅ PlantUML code saved to artifacts/prd_analyzer_component_diagram.puml")

# Try to render the diagram with error handling
if component_puml:
    try:
        render_plantuml_diagram(component_puml, "artifacts/prd_analyzer_component_diagram.png")
        print("✅ Diagram rendered successfully!")
    except Exception as e:
        print(f"⚠️  PlantUML rendering failed: {e}")
        print("📄 You can still use the saved .puml file with:")
        print("   - Online PlantUML editor: http://www.plantuml.com/plantuml/")
        print("   - VS Code PlantUML extension")
        print("   - Local PlantUML installation")

--- Generating Component Diagram for AI-Powered Requirement Analyzer ---

--- Generated PlantUML Code ---
@startuml
!define RECTANGLE class

' COMPONENTS
RECTANGLE "Web Frontend\n(React/Vue.js)" as WebFrontend <<Component>>
RECTANGLE "FastAPI Backend Server" as Backend <<Component>>
RECTANGLE "AI Processing Engine" as AIEngine <<Component>>
RECTANGLE "Background Job Queue" as JobQueue <<Component>>
database "SQLite Database" as Database
folder "File Storage System" as FileStorage
cloud "External AI APIs\n(OpenAI/Anthropic/Google)" as ExternalAI

' RELATIONSHIPS

' User interacts with Web Frontend
actor User
User --> WebFrontend : Interacts via browser

' Web Frontend <-> Backend
WebFrontend --(HTTPS)-> Backend : REST API calls\n(User Auth, Conversations,\nPRD ops, File Uploads)

' Backend <-> Database
Backend -- Database : Read/Write\n(Conversations, Messages,\nPRD Versions, etc.)

' Backend <-> File Storage
Backend -- FileStorage : Upload/Download\n(Documents, Exports)

' Backend <-> 

### Task 2: Generate a Sequence Diagram

Next, we'll create a **sequence diagram**. This type of diagram is used to model the interactions between objects in a sequential order — it shows *how* components collaborate to complete a task.

#### 💡 Pro-Tip: Prompting for Behavioral Diagrams

When prompting for a behavioral diagram (like a Sequence or Activity diagram), use a narrative style:

1.  **State the Format:** `Generate PlantUML code for a sequence diagram`.
2.  **Define the Participants:** List the actors or components involved in the interaction.
3.  **Tell a Story:** Describe the sequence of events step-by-step, as if you were narrating the process. Use phrases like `"First, User A does X. Then, System B sends a message to System C."`. This chronological description maps perfectly to the structure of a sequence diagram.

In [ ]:
# Update sequence diagram to be specific to our PRD application
# Use the context from PRD and schema for a realistic workflow
sequence_diagram_prompt = f"""
You are an expert system architect. Generate PlantUML code for a sequence diagram illustrating the process of a user submitting a problem statement and receiving an AI-generated PRD in the AI-Powered Requirement Analyzer application.

CONTEXT - Product Requirements Document:
{prd_content}

CONTEXT - Database Schema:
{schema_content}

The participants in the diagram should be: 'User', 'Web_Frontend', 'FastAPI_Backend', 'Background_Job_Queue', 'AI_Processing_Engine', 'External_AI_API', and 'SQLite_Database'.

The sequence of events for submitting a problem statement and generating a PRD is as follows:
1. First, User enters a problem statement into the Web Frontend and clicks "Analyze"
2. The Web Frontend sends the problem statement to FastAPI Backend via HTTPS REST API
3. FastAPI Backend stores the new conversation and initial message in SQLite Database
4. FastAPI Backend queues an AI analysis job to the Background Job Queue
5. Background Job Queue delegates the analysis task to AI Processing Engine
6. AI Processing Engine sends the problem statement to External AI API for analysis
7. External AI API returns the analyzed requirements and generated PRD content
8. AI Processing Engine stores the generated PRD version in SQLite Database via FastAPI Backend
9. AI Processing Engine updates the conversation status to "completed"
10. FastAPI Backend notifies Web Frontend that the PRD is ready
11. Web Frontend displays the generated PRD to the User

Output only the raw PlantUML code inside a markdown block.
"""

print("--- Generating Sequence Diagram for AI-Powered Requirement Analyzer ---")
sequence_puml_raw = get_completion(sequence_diagram_prompt, client, model_name, api_provider)
sequence_puml = clean_llm_output(sequence_puml_raw, language='plantuml')

print("\n--- Generated PlantUML Code ---")
print(sequence_puml)

# Save the PlantUML code to a file for manual rendering if needed
if sequence_puml:
    save_artifact(sequence_puml, "artifacts/prd_analyzer_sequence_diagram.puml", overwrite=True)
    print("\n✅ PlantUML code saved to artifacts/prd_analyzer_sequence_diagram.puml")

# Try to render the diagram with error handling
if sequence_puml:
    try:
        render_plantuml_diagram(sequence_puml, "artifacts/prd_analyzer_sequence_diagram.png")
        print("✅ Sequence diagram rendered successfully!")
    except Exception as e:
        print(f"⚠️  PlantUML rendering failed: {e}")
        print("📄 You can still use the saved .puml file with:")
        print("   - Online PlantUML editor: http://www.plantuml.com/plantuml/")
        print("   - VS Code PlantUML extension")
        print("   - Local PlantUML installation")
        
# Alternative: Try a simple manual PNG creation approach if PlantUML server fails
print("\n🔧 Alternative approach: Use local PlantUML if available")
print("   If you have PlantUML locally installed, run:")
print(f"   java -jar plantuml.jar artifacts/prd_analyzer_sequence_diagram.puml")

### 🔧 PlantUML Rendering Solutions

If the automatic PlantUML rendering fails (as it sometimes does due to server issues), here are several reliable alternatives:

#### **Option 1: VS Code PlantUML Extension (Recommended)**
1. Install the PlantUML extension in VS Code (by jebbs)
2. Open the saved `.puml` files in the `artifacts` folder
3. Use `Ctrl+Shift+P` → "PlantUML: Preview Current Diagram"
4. Export as PNG, SVG, or other formats

#### **Option 2: Online PlantUML Editor**
1. Visit: http://www.plantuml.com/plantuml/
2. Copy and paste the PlantUML code from the output above
3. The diagram will render automatically
4. Download the image using the export options

#### **Option 3: Local PlantUML Installation**
1. Download PlantUML JAR: https://plantuml.com/download
2. Ensure Java is installed on your system
3. Run: `java -jar plantuml.jar your-diagram-file.puml`

#### **Option 4: Alternative Diagramming Tools**
- **Mermaid Live Editor**: https://mermaid.live/
- **Draw.io**: https://app.diagrams.net/
- **Lucidchart**: https://www.lucidchart.com/

The key learning objective is achieved regardless of rendering method - you've successfully used AI to generate diagram-as-code from structured requirements!

## Lab Conclusion

Great work! In this practice lab, you have reinforced your ability to use an LLM to generate different types of architectural diagrams from a text description. You created both a static component diagram showing the system's structure and a dynamic sequence diagram showing a key user interaction. This skill is invaluable for quickly communicating complex designs to your team.